In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
from eumap.misc import find_files, nan_percentile, GoogleSheet, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.mapper import SpaceOverlay
import geopandas as gpd
from pathlib import Path
from minio import Minio
import rasterio
import pyproj
from shapely.geometry import Point
import requests
import warnings
warnings.filterwarnings('default')

# os.environ['PROJ_LIB'] = '/opt/conda/share/proj'
folder = '/mnt/inca/soc_eu_model'
# /home/opengeohub/.local/bin

/home/opengeohub/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### check if what is need to be overlayed

In [2]:
from shapely.geometry import Point
from geopandas import gpd

# def get_data_to_be_overlayed(whole=False):
#     df4326 = gpd.read_file(f'{folder}/data/soil_overlay.4326.gpkg')
#     df3035 = gpd.read_file(f'{folder}/data/soil_overlay.3035.gpkg')
    
#     keys = ['sample_id', 'lat', 'lon', 'time', 'hzn_top', 'hzn_btm', 'ref']
#     if whole:
#         return df4326,df3035
#     else:
#         new4326 = gpd.read_file(f'{folder_path}/data/soil_overlay.4326_v2.gpkg')
#         merge4326 = pd.merge(df4326, new4326, on=keys, how='outer', indicator=True)
#         different_4326 = merge4326[merge4326['_merge'] != 'both']
        
#         new3035 = gpd.read_file(f'{folder_path}/data/soil_overlay.3035_v2.gpkg')
#         merge3035 = pd.merge(df3035, new3035, on=keys, how='outer', indicator=True)
#         different_3035 = merge3035[merge3035['_merge'] != 'both']
#         return different_4326,different_3035

# df4326, df3035 = get_data_to_be_overlayed(whole=True)     

# # create gpkg

df = pd.read_csv(f'{folder}/data/000_soil.full_qa.controlled.csv', low_memory=False)
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

df4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
df3035 = df4326.to_crs("EPSG:3035")


### generate overlay links
- read in the files specified by Google Sheet
- convert the files into readable linkes for overlay

In [3]:
# read in potential usable overlay files
key_file = '/mnt/inca/soc_eu_model/gaia-319808-913d36b5fca4.json'
url = 'https://docs.google.com/spreadsheets/d/1eIoPAvWM5jrhLrr25jwguAIR0YxOh3f5-CdXwpcOIz8/edit#gid=0'

gsheet = GoogleSheet(key_file, url)
covar = gsheet.covar

/mnt/inca/soc_eu_model/SOC-EU/code/src/eumap/eumap/misc.py:363: ResourceWarning: unclosed <ssl.SSLSocket fd=61, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.5', 52296), raddr=('142.250.179.170', 443)>
  self._read_gsheet()


In [4]:
# function to generate file paths by year, and check if the urls are valid
def generate_overlay_path(row,year):
    # determine if static variable
    if row['temporal resolution'] == 'static':
        return [row['path']],[row['path']]
    
    # determine if the year is ahead of the availibility of the variable
    if year>int(row['end year']):
        year = int(row['end year'])
    
    # determine if it's an annual variable or (bi)monthly variable
    if '{start_m}' not in row['path']:
        output_paths = [row['path'].replace('{year}',f'{int(year)}')]
    else:
        output_paths = []
        start_list = row['start_m'].split(', ')
        end_list = row['end_m'].split(', ')
        output_paths = [row['path'].replace('{year}',f'{int(year)}').replace('{start_m}',start_list[i]).replace('{end_m}',end_list[i]) for i in range(len(end_list))]
    
    if '{perc}' in row['path']:
        perc_list = row['perc'].split(',')
        output_paths = [p.replace('{perc}', perc) for p in output_paths for perc in perc_list]
        
    if (row['leap year'] == '1') & (year%4==0):
        output_paths = [p.replace('0228', '0229') if '0228' in p else p for p in output_paths]
    
    return output_paths, [i.replace(str(int(year)),'{year}') for i in output_paths]
    
def check_path(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        # Check if the status code is not 200 (OK). You might want to specifically check for 404 or other error codes.
        if response.status_code == 404:
            print(f"{url} returned HTTP 404 Not Found")
            return url
        elif response.status_code != 200:
            print(f"{url} returned HTTP {response.status_code}")
            return url
        return None  # URL is fine (HTTP 200), or you might want to handle redirections (HTTP 3xx) separately if needed.
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {str(e)}")
        return url
    
# # check function validity
# # generate paths
# paths = []
# for index,row in covar.iterrows():
#     paths.extend(generate_overlay_path(row,2000))
    
pathl = []
namel = []
year = 2022
for index,row in covar.iterrows():
    paths, names = generate_overlay_path(row, year)
    pathl.extend(paths)
    namel.extend(names)
    
for i in pathl:
    check_path(i)

### overlay year by year
- divide soil data by year
- overlay the soil data in each year with corresponding covariates

In [5]:
import warnings
warnings.filterwarnings("ignore")

# epsg 3035 overlay
co3035 = covar.loc[covar['epsg']=='3035']

for year in np.arange(2000,2012,1):
    pathl = []
    namel = []
    for index,row in co3035.iterrows():
        paths, names = generate_overlay_path(row, year)
        pathl.extend(paths)
        namel.extend(names)
        # path3035.extend(generate_overlay_path(row,year))
    
    path_stem = [i.split('/')[-1][0:-4] for i in pathl]
    namel = [i.split('/')[-1][0:-4] for i in namel]
    name_mapping = dict(zip(path_stem,namel))
    
    df_overlay = df3035.loc[df3035['time']==year]
    if len(df_overlay)==0:
        print(f'no data for year {year}')
        continue
        
    ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}')
    pathl = [Path(ii) for ii in pathl]
    dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
    temp = dfo.run()
    
    temp = temp.rename(columns=name_mapping)
    temp=temp.drop(columns=['overlay_id'])
    # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_3035.csv',index=False)
    temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_3035.csv',index=False)
    ttprint(f'finish overlaying for year {str(int(year))}')

[19:07:22] start overlaying for year 2000, size: 26304
[19:11:29] finish overlaying for year 2000
[19:11:29] start overlaying for year 2001, size: 12777
[19:16:17] finish overlaying for year 2001
[19:16:17] start overlaying for year 2002, size: 11718
[19:21:15] finish overlaying for year 2002
[19:21:15] start overlaying for year 2003, size: 13497
[19:28:05] finish overlaying for year 2003
[19:28:05] start overlaying for year 2004, size: 7649
[19:32:59] finish overlaying for year 2004
[19:32:59] start overlaying for year 2005, size: 12987
[19:38:16] finish overlaying for year 2005
[19:38:16] start overlaying for year 2006, size: 13075
[19:42:40] finish overlaying for year 2006
[19:42:40] start overlaying for year 2007, size: 18561
[19:55:47] finish overlaying for year 2007
[19:55:47] start overlaying for year 2008, size: 11983
[20:30:34] finish overlaying for year 2008
[20:30:34] start overlaying for year 2009, size: 25942
[21:26:58] finish overlaying for year 2009
[21:26:58] start over

In [6]:
import warnings
warnings.filterwarnings("ignore")

# epsg 4326 overlay
co4326 = covar.loc[covar['epsg']=='4326']
path_ori = [i.split('/')[-1][0:-4] for i in co4326['path']]

for year in np.arange(2000,2024,1):
    pathl = []
    namel = []
    for index,row in co4326.iterrows():
        paths, names = generate_overlay_path(row, year)
        pathl.extend(paths)
        namel.extend(names)
        # path4326.extend(generate_overlay_path(row,year))
        
#     for i in pathl:
#         check_path(i)
        
    path_stem = [i.split('/')[-1][0:-4] for i in pathl]
    namel = [i.split('/')[-1][0:-4] for i in namel]
    name_mapping = dict(zip(path_stem,namel))
    
    df_overlay = df4326.loc[df4326['time']==year]
    if len(df_overlay)==0:
        print(f'no data for year {year}')
        continue
        
    ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}')
    pathl = [Path(ii) for ii in pathl]
    dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
    temp = dfo.run()
    
    temp = temp.rename(columns=name_mapping)
    temp=temp.drop(columns=['overlay_id'])
    # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_4326.csv',index=False)
    temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',index=False)
    ttprint(f'finish overlaying for year {str(int(year))}')


[17:57:55] start overlaying for year 2000, size: 26304
[17:58:24] finish overlaying for year 2000
[17:58:24] start overlaying for year 2001, size: 12777
[17:58:45] finish overlaying for year 2001
[17:58:45] start overlaying for year 2002, size: 11718
[17:59:06] finish overlaying for year 2002
[17:59:06] start overlaying for year 2003, size: 13497
[17:59:28] finish overlaying for year 2003
[17:59:28] start overlaying for year 2004, size: 7649
[17:59:46] finish overlaying for year 2004
[17:59:46] start overlaying for year 2005, size: 12987
[18:00:05] finish overlaying for year 2005
[18:00:05] start overlaying for year 2006, size: 13075
[18:00:32] finish overlaying for year 2006
[18:00:32] start overlaying for year 2007, size: 18561
[18:01:04] finish overlaying for year 2007
[18:01:04] start overlaying for year 2008, size: 11983
[18:01:44] finish overlaying for year 2008
[18:01:44] start overlaying for year 2009, size: 25942
[18:02:22] finish overlaying for year 2009
[18:02:22] start over

### assemble the overlayed annual datasets
- read in the overlayed soil data (with covariates) for each year
- combine them into a whole dataset

In [6]:
## read in 3035 datasets
tl = []
for year in np.arange(2000,2023,1):
    temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_3035.csv',low_memory=False)
    print(f'{year}, {len(temp.columns)}, {len(temp)}')
    tl.append(temp)


df3035 = pd.concat(tl)
print(f'whole 3035, {len(df3035.columns)}, {len(df3035)}')

# read in 4326 datasets
tl = []
name_mapping = {'wv_mcd19a2v061.seasconv_m_1km_s_{year}0201_{year}0229_go_epsg.4326_v20230619':'wv_mcd19a2v061.seasconv_m_1km_s_{year}0201_{year}0228_go_epsg.4326_v20230619',
                'wv_mcd19a2v061.seasconv_sd_1km_s_{year}0201_{year}0229_go_epsg.4326_v20230619':'wv_mcd19a2v061.seasconv_sd_1km_s_{year}0201_{year}0228_go_epsg.4326_v20230619'}

for year in np.arange(2000,2023,1):
    temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',low_memory=False)
    temp = temp.rename(columns=name_mapping)
    print(f'{year}, {len(temp.columns)}, {len(temp)}')
    tl.append(temp)

df4326 = pd.concat(tl)
print(f'whole 4326, {len(df4326.columns)}, {len(df4326)}')

2000, 264, 26304
2001, 264, 12777
2002, 264, 11718
2003, 264, 13497
2004, 264, 7649
2005, 264, 12987
2006, 264, 13075
2007, 264, 18561
2008, 264, 11983
2009, 264, 25942
2010, 264, 12330
2011, 264, 15374
2012, 264, 26062
2013, 264, 21428
2014, 264, 16922
2015, 264, 33628
2016, 264, 16177
2017, 264, 15843
2018, 264, 42918
2019, 264, 15088
2020, 264, 10200
2021, 264, 10886
2022, 264, 3294
whole 3035, 264, 394643
2000, 197, 26304
2001, 197, 12777
2002, 197, 11718
2003, 197, 13497
2004, 197, 7649
2005, 197, 12987
2006, 197, 13075
2007, 197, 18561
2008, 197, 11983
2009, 197, 25942
2010, 197, 12330
2011, 197, 15374
2012, 197, 26062
2013, 197, 21428
2014, 197, 16922
2015, 197, 33628
2016, 197, 16177
2017, 197, 15843
2018, 197, 42918
2019, 197, 15088
2020, 197, 10200
2021, 197, 10886
2022, 197, 3294
whole 4326, 197, 394643


In [7]:
## merge
cols3035 = df3035.columns
cols4326 = df4326.columns
shared_columns = cols3035.intersection(cols4326)
shared_columns

# # merge to merge
# cols_list = df3035.columns.values.tolist()
# meta_list = ['lat', 'lon', 'oc', 'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay','silt', 'sand', 'caco3','N',
#              'K', 'P', 'CEC', 'EC', 'nuts0', 'time','hzn_top','hzn_btm','ref','sample_id']
# dff = pd.merge(df3035,df4326,on = meta_list, how='inner')

# concat to merge
cols_list = df3035.columns.values.tolist()
concat_list = [i for i in cols_list if i not in shared_columns]
df3035_2 = df3035[concat_list]

dff = pd.concat([df4326,df3035_2],axis=1)

In [8]:
### check covariates availability
drop_list = []
for col in dff.columns:
    if col in shared_columns:
        continue
    if (dff[col].isna().sum()/len(dff))>0.02:
        print(col, dff[col].isna().sum())
        drop_list.append(col)
        
print(f'remove covariates with more than 2% data unavailable')
dff = dff.drop(columns = drop_list)

lcv_wilderness_li2022.human.footprint_p_1km_s0..0cm_{year}_v16022022 394643
bioclim.var_chelsa.gddlgd0_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 342902
bioclim.var_chelsa.gdgfgd0_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 342902
bioclim.var_chelsa.gddlgd5_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 36851
bioclim.var_chelsa.gdgfgd5_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 36851
bioclim.var_chelsa.lgd_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 258826
bioclim.var_chelsa.swe_m_1km_s_19810101_20101231_eu_epsg.3035_v20230822 343069
remove covariates with more than 2% data unavailable


In [9]:
dff.to_csv(f'{folder}/data/test_covar_overlayed.csv',index=False)

### Assign spatial blocking ID

In [10]:
# create a tiling system first
from eumap.parallel import TilingProcessing
from pathlib import Path
import rasterio
from shapely.geometry import box
import numpy as np
import pandas as pd
import geopandas as gpd

# raster_layer_fn = f'http://192.168.1.30:8333/ai4sh-landmasked/ndvi/ndvi_glad.landast.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127.tif'
# ds = rasterio.open(raster_layer_fn)
# tiles_size = ds.transform[0] * 1000 # 30m -> 30km

# tiling_system = TilingProcessing.generate_tiles(tiles_size, extent=ds.bounds, crs=ds.crs, raster_layer_fn=raster_layer_fn)
# tiling_system = tiling_system.to_crs("EPSG:4326")
# tiling_system = tiling_system[['tile_id','geometry']]
# tiling_system.to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg',  driver="GPKG")
# # tiling_system[tiling_system['raster_mode_count'] > 0].to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg.gpkg',  driver="GPKG")

df = pd.read_csv(f'{folder}/data/test_covar_overlayed.csv',low_memory=False)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

from shapely.geometry import Point
tiles = gpd.read_file(f'{folder}/data/000_tile_eu4326.gpkg')

gdf.crs = tiles.crs
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['index_right','geometry'])


In [21]:
joined_gdf.to_csv('/mnt/inca/soc_eu_model/data/test_covar_overlayed.csv',index=False)